In [1]:
import numpy as np

In [3]:
def Matrix(*a):
    if len(a)==1 and isinstance(a[0], np.ndarray):
        a = a[0]
    return np.array([[float(x) for x in r] for r in a])

def Vector(*a):
    if len(a)==1 and isinstance(a[0], np.ndarray):
        a = a[0]
    return np.array([float(x) for x in a]).reshape(-1,1)

# Black magic
from IPython.display import Latex, SVG, display
from IPython.core.interactiveshell import InteractiveShell

def ndarray_to_latex(arr): 
    if len(arr.shape)==1: 
        arr=arr.reshape(1,-1)
    if len(arr.shape) == 2:
        if max(arr.shape) > 30:
            return None
        str_arr = np.vectorize("{:.3f}".format)(arr)
        return r'\begin{{pmatrix}}{}\end{{pmatrix}}'.format(r'\\ '.join(map('&'.join, str_arr))) 
    if len(arr.shape) == 3 and arr.shape[2]==1:
        if max(arr.shape) > 30:
            return None
        arr = arr[:,:,0]
        str_arr = np.vectorize("{:.3f}".format)(arr)
        return r'\begin{{bmatrix}}[{}]\end{{bmatrix}}'.format(
            r']\\ ['.join(map('&'.join, str_arr))) 
    return None
sh = InteractiveShell.instance()
sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=ndarray_to_latex

## Feedforward Network
一樣有輸入 x, 輸出  y。 但是中間預測、計算的樣子有點不同。
<img src="https://upload.wikimedia.org/wikipedia/en/5/54/Feed_forward_neural_net.gif" />




### 模型是這樣的
一樣考慮輸入是四維向量，輸出有 3 個類別。

我們的輸入 $x=\begin{pmatrix} x_0 \\ x_1 \\ x_2 \\ x_3 \end{pmatrix} $ 是一個向量，我們看成 column vector 好了

### 第 0 層
而 Weight: $ 
W^{(0)} = \begin{pmatrix} W^{(0)}_0 \\ W^{(0)}_1 \\ W^{(0)}_2 \\ W^{(0)}_3 \\ W^{(0)}_4 \\ W^{(0)}_5 \end{pmatrix} =  
\begin{pmatrix} 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}\\ 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}\\ 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}\\ 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}\\ 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}\\ 
W^{(0)}_{0,0} & W^{(0)}_{0,1} &  W^{(0)}_{0,2} & W^{(0)}_{0,3}
 \end{pmatrix} $
 
 Bias: $b^{(0)}=\begin{pmatrix} b^{(0)}_0 \\ b^{(0)}_1 \\ b^{(0)}_2  \\ b^{(0)}_3 \\ b^{(0)}_4 \\ b^{(0)}_5 \end{pmatrix} $ 


我們先計算"線性輸出"  $ c^{(0)} = \begin{pmatrix} c^{(0)}_0 \\ c^{(0)}_1 \\ c^{(0)}_2  \\ c^{(0)}_3 \\ c^{(0)}_4 \\ c^{(0)}_5 \end{pmatrix} =  W^{(0)}x+b^{(0)} =
\begin{pmatrix} W^{(0)}_0 x + b^{(0)}_0 \\ W^{(0)}_1 x + b^{(0)}_1 \\ W^{(0)}_2 x + b^{(0)}_2 \\
W^{(0)}_3 x + b^{(0)}_3 \\ W^{(0)}_4 x + b^{(0)}_4 \\ W^{(0)}_5 x + b^{(0)}_5  \end{pmatrix}   $， 

然後再將結果逐項對一個非線性的函數 $f$  最後得到一個向量。
 
 $d^{(0)} = \begin{pmatrix} d^{(0)}_0 \\ d^{(0)}_1 \\ d^{(0)}_2  \\ d^{(0)}_3 \\ d^{(0)}_4 \\ d^{(0)}_5 \end{pmatrix} 
 = f({W x + b}) = \begin{pmatrix} f(c^{(0)}_0) \\ f(c^{(0)}_1) \\ f(c^{(0)}_2)  \\ f(c^{(0)}_3) \\ f(c^{(0)}_4) \\ f(c^{(0)}_5) \end{pmatrix} $
 
這裡的 $f$ 常常會用 sigmoid , tanh，或者 ReLU ( https://en.wikipedia.org/wiki/Activation_function )。

### 第 1 層
這裡接到輸出，其實和 softmax regression 一樣。

只是輸入變成 $d^{(0)}$, Weight 和 Bias 現在叫做 $W^{(1)}$ 和 $b^{(1)}$ 

因為維度改變，現在 $W^{(1)}$ 是 3x6 的矩陣。 後面接到的輸出都一樣。

所以線性輸出

### $ c^{(1)} =  W^{(1)} d^{(0)} + b^{(1)} $

### $ d^{(1)} =  e^{c^{(1)}} $



當輸入為 x, 最後的 softmax 預測類別是 i 的機率為
###  $q_i = Predict_{W^{(0)}, W^{(1)}, b^{(0)}, b^{(1)}}(Y=i|x)  = \frac {d^{(1)}_i} {\sum_j d^{(1)}_j}$
### 合起來看，就是 $q = \frac {d^{(1)}} {\sum_j d^{(1)}_j}$

### 問題
如果 $W^{(0)}, W^{(1)}, b^{(0)}, b^{(1)}$ 設定為 $A, C, b, d$ (C, d 與前面無關)

softmax function 用
### $\sigma (\mathbf {z} )_{j}={\frac {e^{z_{j}}}{\sum _k e^{z_{k}}}}$
表示，我們簡化上面的過程成為一個算式。

#### Note
$d^{(0)} = f(W^{(0)}x+b^{(0)})$  will become:  $d^{(0)} = f(Ax+b)$

$d^{(1)}=f(W^{(1)}d^{(0)}+b^{(1)})$  will become:  $d^{(1)}=f(Cf(Ax+b)+d)$

$\sigma(W^{(1)}f(W^{(0)}x+b^{(0)})+b^{(1)})$ will become: $\sigma( Cf(Ax+b)+d )$

In [4]:
from IPython.display import display, Latex
display(Latex("$\sigma (Cf(Ax+b)+d)$"))

<IPython.core.display.Latex object>

### 任務：計算最後的猜測機率 $q$
設定：輸入 4 維， 輸出 3 維， 隱藏層 6 維
* 設定一些權重 $A,b,C,d$ (隨意自行填入，或者用 np.random.randint(-2,3, size=...))
* 設定輸入 $x$ (隨意自行填入，或者用 np.random.randint(-2,3, size=...))
* 自行定義 relu, sigmoid 函數 (Hint: np.maximum)
* 算出隱藏層 $z$
* 自行定義 softmax
* 算出最後的 q

### Initialization
#### Note
x is a 4x1 matrix

$A = W^{(0)}$ is a 6x4 matrix

$b = b^{(0)}$ is a 6x1 matrix

$C = W^{(1)}$ is a 3x6 matrix

$d = b^{(1)}$ is a 3x1 matrix

In [5]:
# 請在這裡計算
np.random.seed(1234)

In [6]:
# 測定權重 A, b, C, d 和 x
x = np.random.randint(-2, 3, size=(4, 1))
A = np.random.randint(-2, 3, size=(6, 4))
b = np.random.randint(-2, 3, size=(6, 1))
C = np.random.randint(-2, 3, size=(3, 6))
d = np.random.randint(-2, 3, size=(3, 1))

In [7]:
# 參考答案，設定權重
display(A)
display(b)
display(C)
display(d)
display(x)

array([[-1, -1, -1,  0],
       [ 1,  2,  2,  0],
       [ 0, -2, -2,  2],
       [-2, -1,  0, -2],
       [ 1,  2,  0,  0],
       [ 1,  1, -2, -1]])

array([[ 1],
       [-2],
       [ 1],
       [ 0],
       [ 1],
       [ 2]])

array([[-1,  1,  1,  1,  0, -1],
       [ 1,  2,  0,  1,  2, -1],
       [ 2, -2,  2, -1, -1,  2]])

array([[-1],
       [ 2],
       [-2]])

array([[ 1],
       [ 2],
       [ 2],
       [-2]])

### activation function
#### ReLU
$f(x)={\begin{cases}0&{\text{for }}x\leq 0\\x&{\text{for }}x>0\end{cases}}$

#### Sigmoid
$f(x)=\sigma (x)={\frac {1}{1+e^{-x}}}$

In [8]:
# 定義 relu, sigmoid 函數
def relu(x):
    return np.maximum(x, 0)
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [9]:
d_relu = relu(A@x + b)
d_sigmoid = sigmoid(A@x + b)

In [10]:
# 參考答案 定義 relu, sigmoid 及計算 z
display(d_relu)
display(d_sigmoid)

array([[0],
       [7],
       [0],
       [0],
       [6],
       [3]])

array([[1.79862100e-02],
       [9.99088949e-01],
       [1.67014218e-05],
       [5.00000000e-01],
       [9.97527377e-01],
       [9.52574127e-01]])

### Softmax
$q={\frac {d^{(1)}}{\sum _j d^{(1)}_{j}}}$

In [11]:
# 定義 softmax 及計算 q
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

In [12]:
q_relu = softmax(C@relu(A@x+b)+d)
q_sigmoid = softmax(C@sigmoid(A@x+b)+d)

In [13]:
# 參考答案 定義 softmax 及計算 q
display(q_relu)
display(q_sigmoid)

array([[2.78946809e-10],
       [1.00000000e+00],
       [1.56288219e-18]])

array([[2.39921970e-03],
       [9.97490855e-01],
       [1.09925441e-04]])

### 練習
設計一個網路:
* 輸入是二進位 0 ~ 15
* 輸出依照對於 3 的餘數分成三類


In [14]:
# Hint 下面產生數字 i 的 2 進位向量
i = 13
x = Vector(i%2, (i>>1)%2, (i>>2)%2, (i>>3)%2)
x

array([[1.],
       [0.],
       [1.],
       [1.]])

In [15]:
# 請在這裡計算
# mod 3
A = Matrix([0,0,0,0], 
           [1,-1,1,-1], 
           [-1,1,-1,1],
           [-10,10,-10,10],
           [10,-10,10,-10],
          )
b = Vector(0.1,0,0,-12,-12)
C = Matrix([1,0,0,0,0], 
           [0,1,0,1,0], 
           [0,0,1,0,1]
          )
d = Vector(0,0,0)
for i in range(16):
    x = Vector(i%2, (i>>1)%2, (i>>2)%2, (i>>3)%2)
    q = softmax(C@relu(A@x+b)+d)
    print("i={}, i%3={}, q={}".format(i, i%3, q.argmax()))

i=0, i%3=0, q=0
i=1, i%3=1, q=1
i=2, i%3=2, q=2
i=3, i%3=0, q=0
i=4, i%3=1, q=1
i=5, i%3=2, q=2
i=6, i%3=0, q=0
i=7, i%3=1, q=1
i=8, i%3=2, q=2
i=9, i%3=0, q=0
i=10, i%3=1, q=1
i=11, i%3=2, q=2
i=12, i%3=0, q=0
i=13, i%3=1, q=1
i=14, i%3=2, q=2
i=15, i%3=0, q=0


In [16]:
# 請在這裡計算
# mod 3 窮舉法XD
A = Matrix([0,0,0,0], 
           [1,1,0,0],
           [0,1,1,0],
           [0,0,1,1],
           [1,0,0,1],
           [1,1,1,1],
           [1,0,0,0],
           [0,0,1,0],
           [0,1,0,1],
           [1,1,1,0],
           [1,0,1,1],
           [0,1,0,0],
           [0,0,0,1],
           [1,0,1,0],
           [1,1,0,1],
           [0,1,1,1]
          )
b = Vector(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
C = Matrix([1/6,1/6,1/6,1/6,1/6,1/6,0,0,0,0,0,0,0,0,0,0], 
           [0,0,0,0,0,0,1/5,1/5,1/5,1/5,1/5,0,0,0,0,0], 
           [0,0,0,0,0,0,0,0,0,0,0,1/5,1/5,1/5,1/5,1/5]
          )
d = Vector(0,0,0)
for i in range(16):
    x = Vector(i%2, (i>>1)%2, (i>>2)%2, (i>>3)%2)
    q = softmax(C@relu(A@x+b)+d)
    print("i={}, i%3={}, q={}".format(i, i%3, q.argmax()))

i=0, i%3=0, q=0
i=1, i%3=1, q=1
i=2, i%3=2, q=2
i=3, i%3=0, q=0
i=4, i%3=1, q=1
i=5, i%3=2, q=1
i=6, i%3=0, q=0
i=7, i%3=1, q=1
i=8, i%3=2, q=2
i=9, i%3=0, q=0
i=10, i%3=1, q=2
i=11, i%3=2, q=2
i=12, i%3=0, q=0
i=13, i%3=1, q=1
i=14, i%3=2, q=2
i=15, i%3=0, q=0


In [17]:
# 請在這裡計算
# mod 4
A = Matrix([-1,-1,0,0],
           [1,-1,0,0],
           [-1,1,0,0],
           [1,1,0,0]
          )
b = Vector(1,0,0,0)
C = Matrix([1,0,0,0], 
           [0,1,0,0], 
           [0,0,1,0],
           [0,0,0,1]
          )
d = Vector(0,0,0,0)
for i in range(16):
    x = Vector(i%2, (i>>1)%2, (i>>2)%2, (i>>3)%2)
    q = softmax(C@relu(A@x+b)+d)
    print("i={}, i%4={}, q={}".format(i, i%4, q.argmax()))

i=0, i%4=0, q=0
i=1, i%4=1, q=1
i=2, i%4=2, q=2
i=3, i%4=3, q=3
i=4, i%4=0, q=0
i=5, i%4=1, q=1
i=6, i%4=2, q=2
i=7, i%4=3, q=3
i=8, i%4=0, q=0
i=9, i%4=1, q=1
i=10, i%4=2, q=2
i=11, i%4=3, q=3
i=12, i%4=0, q=0
i=13, i%4=1, q=1
i=14, i%4=2, q=2
i=15, i%4=3, q=3


### 練習
設計一個網路來判斷井字棋是否有連成直線(只需要判斷其中一方即可):
* 輸入是 9 維向量，0 代表空格，1 代表有下子 
* 輸出是二維(softmax)或一維(sigmoid)皆可，用來代表 True, False

有連線的例子

```
_X_
X__
XXX

XXX
XX_
_XX

__X
_XX
X__
```

沒連線的例子
```
XX_
X__
_XX

_X_
XX_
X_X

__X
_XX
_X_
```

In [18]:
# 請在這裡計算
# 連線的轉置矩陣
A = Matrix([1,1,1,0,0,0,0,0,0],
           [0,0,0,1,1,1,0,0,0],
           [0,0,0,0,0,0,1,1,1],
           [1,0,0,1,0,0,1,0,0],
           [0,1,0,0,1,0,0,1,0],
           [0,0,1,0,0,1,0,0,1],
           [0,0,1,0,1,0,1,0,0],
           [1,0,0,0,1,0,0,0,1]
          )
# 只有連線的組合經過Ax後會等於3，因此將bias設成-2是其他沒有連線的組合經過Ax-b後會小於等於零
b = Vector(-2,-2,-2,-2,-2,-2,-2,-2)
C = Matrix([-1,-1,-1,-1,-1,-1,-1,-1], 
           [1,1,1,1,1,1,1,1]
          )
# 為了讓沒有連線的輸出較大
d = Vector(1,0)

In [19]:
# 測試你的答案
def my_result(x):
    # return 0 means no, 1 means yes
    return (C@relu(A@x+b)+d).argmax()
    # or sigmoid based
    # return (C@relu(A@x+b)+d) > 0

def truth(x):
    x = x.reshape(3,3)
    return (x.all(axis=0).any() or
            x.all(axis=1).any() or
            x.diagonal().all() or
            x[::-1].diagonal().all())

for i in range(512):
    x = np.array([[(i>>j)&1] for j in range(9)])
    assert my_result(x) == truth(x)
print("test passed")

test passed
